# 2.6. Introducing a deeper network

## Data loading and preprocessing

In [3]:
import pandas as pd
from google.colab import files
#uploaded = files.upload()
data = pd.read_csv('/content/preprocessed_data.csv')
data = data.sample(frac=1) # a bit weird
data.head()

,Winner,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_att,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_att,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,...,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_total_rounds_fought,R_total_time_fought(seconds),R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,weight_class_Women's Bantamweight,weight_class_Women's Featherweight,weight_class_Women's Flyweight,weight_class_Women's Strawweight,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch
420,Red,False,3,0.0,1.0,0.0,1.500000,1.500000,3.500000,2.000000,11.500000,5.000000,5.00,4.500000,18.000000,9.500000,0.000000,0.500000,0.500000,1.500000,0.500000,20.000000,11.500000,0.570000,0.000000,1.500000,1.000000,0.330000,59.500000,47.000000,1.0,1.0,10.500000,5.500000,3.5,3.000000,38.00,13.500000,1.5,1.500000,...,120.428571,70.285714,18.0,694.571429,1.0,0.0,0.0,2.0,0.0,3.0,0.0,5.0,180.34,182.88,170.0,33.0,27.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0
545,Red,False,3,1.0,0.0,0.0,7.200000,4.200000,7.600000,4.800000,33.200000,14.800000,3.80,3.800000,36.400000,18.400000,0.000000,1.000000,0.800000,2.200000,0.200000,44.600000,23.400000,0.538000,0.600000,3.200000,1.800000,0.448000,66.800000,41.000000,4.0,1.0,7.000000,3.600000,3.0,1.800000,39.60,11.000000,0.6,0.600000,...,95.307692,46.230769,31.0,599.692308,1.0,0.0,0.0,2.0,3.0,3.0,1.0,9.0,187.96,193.04,205.0,30.0,38.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
2003,Blue,False,3,1.0,0.0,0.0,7.000000,3.000000,4.000000,3.000000,46.000000,25.000000,0.00,0.000000,14.000000,5.000000,0.000000,29.000000,20.000000,0.000000,0.000000,50.000000,28.000000,0.560000,0.000000,4.000000,0.000000,0.000000,57.000000,35.000000,0.0,1.0,7.000000,5.000000,2.0,2.000000,82.00,45.000000,0.0,0.000000,...,25.000000,15.500000,2.0,201.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,190.50,187.96,205.0,28.0,32.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1147,Red,True,5,0.0,1.0,0.0,6.777778,4.333333,4.333333,2.444444,35.777778,14.111111,8.00,5.666667,38.888889,16.111111,0.222222,2.444444,1.777778,0.888889,0.111111,48.111111,22.222222,0.537778,0.222222,1.666667,0.666667,0.333333,58.555556,31.666667,4.0,2.0,4.222222,2.555556,4.0,2.777778,21.00,8.777778,8.0,5.888889,...,88.250000,35.437500,47.0,807.000000,9.0,0.0,0.0,6.0,4.0,5.0,0.0,15.0,193.04,213.36,205.0,33.0,28.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2275,Blue,False,3,2.0,0.0,0.0,18.250000,8.000000,4.250000,2.500000,80.250000,38.000000,0.75,0.250000,52.000000,21.750000,0.500000,15.000000,11.000000,0.000000,0.000000,85.250000,40.750000,0.355000,0.000000,0.000000,0.000000,0.000000,90.250000,44.250000,2.0,2.0,19.000000,9.250000,2.5,1.250000,111.25,28.000000,8.0,5.000000,...,139.769231,63.769231,35.0,787.538462,0.0,0.0,2.0,4.0,1.0,0.0,0.0,7.0,175.26,177.80,155.0,27.0,28.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


In [0]:
data['Winner'] = data['Winner'].map(lambda x: 1 if x == 'Red' else 0)
data['title_bout'] = data['title_bout'].map(lambda x: 1 if x == 'True' else 0)

In [0]:
train_size = int(0.8*len(data))
features = data.drop(columns=['Winner'])
targets = data['Winner']
X_train, X_test = features.values[:train_size, :], features.values[train_size:, :]
y_train, y_test = targets.values[:train_size], targets.values[train_size:]

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
#sns.pairplot(data)

In [0]:
corr = data.corr()
cmap = sns.diverging_palette(250, 10, as_cmap=True)
plt.figure(figsize=(40, 80))
sns.heatmap(corr[['Winner']], cmap=cmap, vmax=.3,
            square=True, linewidths=.5, cbar_kws={"shrink": .5},
            annot=True)

## Tensorflow ANNs

In [9]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1024, activation=tf.nn.leaky_relu), # x if x > 0 else alpha*x 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation=tf.nn.leaky_relu), 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid') # [0, 1]
])

red = len(y_train[y_train>0])
blue = len(y_train) - red
total = len(y_train)
weight_for_red = total / (2 * red) 
weight_for_blue = total / (2 * blue)
class_weight = {0: weight_for_blue, 1: weight_for_red}
print(class_weight)

adam_optimizer = tf.keras.optimizers.Adam()
model.compile(
    optimizer=adam_optimizer,
    loss='binary_crossentropy',
    metrics=[
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
    ]
)

save_best_callback = tf.keras.callbacks.ModelCheckpoint(
    '/content/model-{epoch:02d}-{val_accuracy:.2f}.hdf5', 
    monitor='val_accuracy', 
    verbose=1,
    save_best_only=True, 
    save_weights_only=False,
    save_frequency=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
model.fit(X_train_scaled, 
          y_train,
          class_weight=class_weight,
          batch_size=64,
          validation_split=0.1,
          #callbacks=[save_best_callback],
          epochs=20)

{0: 1.4979144942648592, 1: 0.7505224660397074}
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 2585 samples, validate on 288 samples
Epoch 1/20
2585/2585 [==============================] - 2s 951us/sample - loss: 0.7379 - tp: 1031.0000 - fp: 362.0000 - tn: 498.0000 - fn: 694.0000 - accuracy: 0.5915 - precision: 0.7401 - recall: 0.5977 - auc: 0.6273 - val_loss: 0.7155 - val_tp: 128.0000 - val_fp: 46.0000 - val_tn: 53.0000 - val_fn: 61.0000 - val_accuracy: 0.6285 - val_precision: 0.7356 - val_recall: 0.6772 - val_auc: 0.6410
Epoch 2/20
2585/2585 [==============================] - 1s 221us/sample - loss: 0.6707 - tp: 1041.0000 - fp: 295.0000 - tn: 565.0000 - fn: 684.0000 - accuracy: 0.6213 - precision: 0.7792 - recall: 0.6035 - auc: 0.6857 - val_loss: 0.6706 - val_tp: 76.0000 - val_fp: 24.0000 - val_tn: 75.0000 - val_fn: 113.0000 - val_accuracy: 0.5243 - val_precision: 0.7600 - val_recall: 0.4021 - val_auc: 0.6578
Epoch 3/20
2585/2585 [==============================] - 1s 222us

In [10]:
import numpy as np
#model = tf.keras.models.load_model('/content/model-43-0.64.hdf5')
X_test_scaled = scaler.transform(X_test)
model.evaluate(X_test_scaled, y_test)
#np.round(model.predict(X_test))

719/719 [==============================] - 1s 915us/sample - loss: 0.7715 - tp: 317.0000 - fp: 116.0000 - tn: 137.0000 - fn: 149.0000 - accuracy: 0.6314 - precision: 0.7321 - recall: 0.6803 - auc: 0.6499


[0.7715438687254226,
 317.0,
 116.0,
 137.0,
 149.0,
 0.63143253,
 0.7321016,
 0.6802575,
 0.6498922]

In [2]:
!pip install --upgrade tensorflow

     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 450kB 50.4MB/s 
     |████████████████████████████████| 3.9MB 57.3MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
